In [1]:
# Refactored end-to-end pipeline for Risk-Drift + Adversarial Fragility
# Save as pipeline_refactor.py or run inside a Jupyter notebook cell.

import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from pathlib import Path

# --- CONFIG --- #
DATA_PATH = "Health_Risk_Dataset.csv"
FEATURES = ['Respiratory_Rate','Oxygen_Saturation','O2_Scale','Systolic_BP',
            'Heart_Rate','Temperature','Consciousness','On_Oxygen']
CONTINUOUS_VITALS = ['Respiratory_Rate','Oxygen_Saturation','Systolic_BP','Heart_Rate','Temperature']

SAFE_RANGES = {
    'Respiratory_Rate': (12, 20),
    'Oxygen_Saturation': (94, 100),
    'Systolic_BP': (100, 140),
    'Heart_Rate': (60, 100),
    'Temperature': (36.0, 37.5)
}

CONSCIOUSNESS_MAP = {'A':0,'C':1,'P':2,'V':3,'U':4}
RISK_MAP = {'Normal':0,'Low':1,'Medium':2,'High':3}
INV_RISK_MAP = {v:k for k,v in RISK_MAP.items()}

# --- Load --- #
if not Path(DATA_PATH).exists():
    raise FileNotFoundError(f"{DATA_PATH} not found. Place the CSV in working directory.")
df = pd.read_csv(DATA_PATH, sep=None, engine="python")
df.columns = [c.strip() for c in df.columns]

# quick schema check
expected = {'Patient_ID','Risk_Level'}
if not expected.issubset(set(df.columns)):
    raise ValueError(f"CSV missing required columns. Found: {df.columns.tolist()}")

# --- Preprocess --- #
df = df.reset_index(drop=True)
df['Consciousness'] = df['Consciousness'].map(CONSCIOUSNESS_MAP)
df['Risk_Num'] = df['Risk_Level'].map(RISK_MAP)
df['On_Oxygen'] = pd.to_numeric(df['On_Oxygen'], errors='coerce').fillna(0).astype(int)

# drop rows with missing features (small realistic datasets — alternative: impute)
missing_count = df[FEATURES].isna().sum().sum()
if missing_count > 0:
    print(f"Dropping {df[FEATURES].isna().any(axis=1).sum()} rows with missing feature values.")
    df = df.loc[~df[FEATURES].isna().any(axis=1)].reset_index(drop=True)

X = df[FEATURES].copy()
y = df['Risk_Level'].copy()
y_num = df['Risk_Num'].copy()

# --- Train model --- #
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
# We scale continuous vitals + keep categorical columns as-is for model input scaling only
scaled_cols = CONTINUOUS_VITALS + ['O2_Scale','Consciousness','On_Oxygen']
X_train_scaled = scaler.fit_transform(X_train[scaled_cols])
X_test_scaled = scaler.transform(X_test[scaled_cols])

# Train on raw values (forest can handle different scales) — but we keep scaler for clustering
model = RandomForestClassifier(n_estimators=400, random_state=42)
model.fit(X_train, y_train)
print("RandomForest trained. Test accuracy:", round(model.score(X_test, y_test), 4))

# --- SHAP (optional) --- #
try:
    import shap
    explainer = shap.TreeExplainer(model)
    # compute sample shap values for a subset (speed)
    sample = X_test.sample(min(200, len(X_test)), random_state=1)
    shap_values = explainer.shap_values(sample)
    SHAP_AVAILABLE = True
    print("SHAP available.")
except Exception as e:
    SHAP_AVAILABLE = False
    print("SHAP not available:", e)

# --- Clustering (GMM) & Risk Drift --- #
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

cluster_input = scaler.transform(X[scaled_cols])
best_k = 4; best_score = -1
for k in range(3,9):
    g = GaussianMixture(n_components=k, random_state=42).fit(cluster_input)
    labels = g.predict(cluster_input)
    try:
        s = silhouette_score(cluster_input, labels)
    except Exception:
        s = -1
    if s > best_score:
        best_score = s; best_k = k

gmm = GaussianMixture(n_components=best_k, random_state=42).fit(cluster_input)
df['Cluster'] = gmm.predict(cluster_input)
centroids = gmm.means_
print(f"GMM chosen clusters: {best_k} (silhouette {best_score:.3f})")

cluster_risk = df.groupby('Cluster')['Risk_Num'].mean().reindex(range(best_k)).fillna(0).values
dist_matrix = np.linalg.norm(centroids[:, None] - centroids[None, :], axis=2)
similarity = np.exp(-dist_matrix)
risk_drift_cluster = []
for a in range(best_k):
    higher = cluster_risk > cluster_risk[a]
    if higher.sum()==0:
        risk_drift_cluster.append(0.0)
    else:
        risk_drift_cluster.append(float(similarity[a,higher].sum() / similarity[a].sum()))
risk_drift_cluster = np.array(risk_drift_cluster)
df['Risk_Drift'] = df['Cluster'].map(lambda c: float(risk_drift_cluster[c]))
df['Risk_Drift_Score'] = (100*df['Risk_Drift']).round(1)

# --- Probabilistic deterioration (KDE-based) + Monte Carlo --- #
from sklearn.neighbors import KernelDensity

kde_models = {}
for vital in CONTINUOUS_VITALS:
    vals = df[vital].values.reshape(-1,1)
    low, high = SAFE_RANGES[vital]
    residuals = []
    for v in vals.flatten():
        if v < low:
            residuals.append((low - v))
        elif v > high:
            residuals.append((v - high))
        else:
            residuals.append(abs(np.random.normal(loc=0.2, scale=0.12)))
    residuals = np.array(residuals).reshape(-1,1)
    try:
        kde = KernelDensity(kernel='gaussian', bandwidth=0.3).fit(np.log1p(residuals))
        kde_models[vital] = ('kde', kde)
    except Exception:
        mu, sigma = float(residuals.mean()), float(residuals.std()) if residuals.std()>1e-6 else 0.5
        kde_models[vital] = ('norm', (mu, sigma))

def sample_deterioration_single(base_row):
    simulated = base_row.copy()
    VITAL_BOUNDS = {
        'Respiratory_Rate': (4, 60),
        'Oxygen_Saturation': (40, 100),
        'Systolic_BP': (50, 220),
        'Heart_Rate': (30, 220),
        'Temperature': (34.0, 42.0)
    }
    for vital in CONTINUOUS_VITALS:
        low, high = SAFE_RANGES[vital]
        orig = float(base_row[vital])
        typ = kde_models[vital][0]
        if typ == 'kde':
            kde = kde_models[vital][1]
            sampled = np.expm1(kde.sample(1)).flatten()[0]
            sampled = max(0.01, float(sampled) * np.random.uniform(0.6,1.4))
        else:
            mu, sigma = kde_models[vital][1]
            sampled = max(0.01, np.random.normal(mu, sigma))
        if vital == "Oxygen_Saturation":
            new_val = orig - sampled
        elif vital == "Respiratory_Rate":
            new_val = orig + sampled
        elif vital == "Heart_Rate":
            new_val = orig + sampled * np.random.uniform(0.9,1.3)
        elif vital == "Systolic_BP":
            if orig < low:
                new_val = orig - sampled
            elif orig > high:
                new_val = orig + sampled * np.random.uniform(0.4,1.2)
            else:
                new_val = orig - sampled * np.random.uniform(0.2,0.9)
        elif vital == "Temperature":
            new_val = orig + sampled * np.random.uniform(0.2,0.6)
        lo, hi = VITAL_BOUNDS[vital]
        simulated[vital] = float(np.clip(new_val, lo, hi))
    return simulated

def monte_carlo_future_prob(X_df, n_sims=200, required_label="High"):
    results = []
    high_idx = list(model.classes_).index(required_label)
    for idx, row in X_df.iterrows():
        base_row = row.copy()
        probs_high = []
        for s in range(n_sims):
            sim_row = sample_deterioration_single(base_row)
            prob_high = model.predict_proba(pd.DataFrame([sim_row[FEATURES]]))[0][high_idx]
            probs_high.append(prob_high)
        probs = np.array(probs_high)
        results.append({
            'Patient_idx': idx,
            'Patient_ID': df.loc[idx,'Patient_ID'],
            'current_high_prob': model.predict_proba(pd.DataFrame([base_row[FEATURES]]))[0][high_idx],
            'future_high_prob_mean': float(probs.mean()),
            'future_high_prob_p95': float(np.percentile(probs,95)),
            'future_high_prob_p05': float(np.percentile(probs,5)),
            'future_high_prob_std': float(probs.std())
        })
    return pd.DataFrame(results)

# run Monte Carlo on a sample (speed). Increase n_sims or sample size if time available.
print("Running Monte Carlo sample (this can take a while)...")
mc_sample = monte_carlo_future_prob(X.sample(min(200,len(X)), random_state=1), n_sims=200)
print("Monte Carlo sample done.")

# --- Adversarial minimal perturbation (Nelder-Mead) w/ fallback random search --- #
try:
    from scipy.optimize import minimize
    SCIPY_AVAILABLE = True
except Exception:
    SCIPY_AVAILABLE = False

VITAL_BOUNDS = {
    'Respiratory_Rate': (4, 60),
    'Oxygen_Saturation': (40, 100),
    'Systolic_BP': (50, 220),
    'Heart_Rate': (30, 220),
    'Temperature': (34.0, 42.0)
}

def get_row_features_for_model(row):
    return row[FEATURES]

def adversarial_minimal_change_for_row(idx, required_prob=0.5, target_label="High", n_restarts=8, maxiter=300):
    base_full = df.loc[idx].copy()
    base = base_full[FEATURES].copy()
    base_vals = np.array([base[v] for v in CONTINUOUS_VITALS], dtype=float)
    target_index = list(model.classes_).index(target_label)
    lower = np.array([VITAL_BOUNDS[v][0] for v in CONTINUOUS_VITALS])
    upper = np.array([VITAL_BOUNDS[v][1] for v in CONTINUOUS_VITALS])

    def build_input(x):
        row_copy = base.copy()
        for i,v in enumerate(CONTINUOUS_VITALS):
            row_copy[v] = float(x[i])
        return row_copy

    def obj(x):
        if np.any(x < lower) or np.any(x > upper):
            return 1e6 + np.linalg.norm(x - base_vals)
        prob = model.predict_proba(pd.DataFrame([build_input(x)]))[0][target_index]
        dist = np.linalg.norm(x - base_vals)
        penalty = 200.0 * max(0.0, required_prob - prob)
        return dist + penalty

    best = {'success': False, 'dist': np.inf, 'prob': None, 'x': None}
    if SCIPY_AVAILABLE:
        for r in range(n_restarts):
            x0 = np.clip(base_vals + np.random.normal(scale=0.5, size=len(base_vals)), lower, upper)
            res = minimize(obj, x0, method='Nelder-Mead', options={'maxiter':maxiter, 'xatol':1e-3, 'fatol':1e-3, 'disp': False})
            x_opt = np.clip(res.x, lower, upper)
            prob = model.predict_proba(pd.DataFrame([build_input(x_opt)]))[0][target_index]
            dist = float(np.linalg.norm(x_opt - base_vals))
            if prob >= required_prob and dist < best['dist']:
                best.update({'success': True, 'dist': dist, 'prob': float(prob), 'x': x_opt.copy()})
    else:
        # Random-search fallback
        for t in range(3000):
            x_try = np.clip(base_vals + np.random.normal(scale=1.0, size=len(base_vals)), lower, upper)
            prob = model.predict_proba(pd.DataFrame([build_input(x_try)]))[0][target_index]
            dist = float(np.linalg.norm(x_try - base_vals))
            if prob >= required_prob and dist < best['dist']:
                best.update({'success': True, 'dist': dist, 'prob': float(prob), 'x': x_try.copy()})

    # if we didn't find successful flip, provide best prob found:
    if not best['success']:
        best_prob = -1; best_x = None
        for t in range(800):
            x_try = np.clip(base_vals + np.random.normal(scale=1.2, size=len(base_vals)), lower, upper)
            prob = model.predict_proba(pd.DataFrame([build_input(x_try)]))[0][target_index]
            if prob > best_prob:
                best_prob = float(prob); best_x = x_try.copy()
        best.update({'success': False, 'dist': float(np.linalg.norm(best_x - base_vals)), 'prob': float(best_prob), 'x': best_x})

    deltas = {v: float(best['x'][i] - base_vals[i]) for i,v in enumerate(CONTINUOUS_VITALS)} if best['x'] is not None else None
    new_vals = {v: float(best['x'][i]) for i,v in enumerate(CONTINUOUS_VITALS)} if best['x'] is not None else None

    return {
        'Patient_idx': idx, 'Patient_ID': df.loc[idx,'Patient_ID'],
        'success': best['success'], 'distance': best['dist'],
        'prob_high': best['prob'], 'deltas': deltas, 'new_values': new_vals
    }

# Run adversarial search on a sample (speed)
sample_idxs = list(X.sample(min(60,len(X)), random_state=2).index)
adv_results = [adversarial_minimal_change_for_row(i, required_prob=0.5, n_restarts=8) for i in sample_idxs]
adv_df = pd.DataFrame(adv_results)
adv_df_sorted = adv_df.sort_values(by=['success','distance','prob_high'], ascending=[False, True, False])

# Combine Monte Carlo and adversarial results for the same sampled patients
combined = pd.merge(
    adv_df, 
    mc_sample, 
    on=['Patient_idx', 'Patient_ID'],  # Join on both index and Patient_ID
    how='left'
)
combined['Risk_Drift_Score'] = combined['Patient_idx'].map(lambda i: df.loc[i,'Risk_Drift_Score'])
combined['Current_Risk'] = combined['Patient_idx'].map(lambda i: df.loc[i,'Risk_Level'])

# Save outputs
combined.to_csv("combined_fragility_and_mc_summary.csv", index=False)
adv_df_sorted.to_csv("adversarial_summary_sample.csv", index=False)
mc_sample.to_csv("monte_carlo_summary_sample.csv", index=False)

print("Saved files: combined_fragility_and_mc_summary.csv, adversarial_summary_sample.csv, monte_carlo_summary_sample.csv")
print("Top fragile patients (adversarial flip found with small distance):")
print(adv_df_sorted[adv_df_sorted['success']].sort_values('distance').head(10))

# If in a Jupyter environment, display small samples
try:
    from IPython.display import display
    display(adv_df_sorted.head(20))
    display(mc_sample.head(20))
    display(combined.head(20))
except Exception:
    pass

# End of pipeline

RandomForest trained. Test accuracy: 0.96
SHAP available.
GMM chosen clusters: 3 (silhouette 0.364)
Running Monte Carlo sample (this can take a while)...
Monte Carlo sample done.
Saved files: combined_fragility_and_mc_summary.csv, adversarial_summary_sample.csv, monte_carlo_summary_sample.csv
Top fragile patients (adversarial flip found with small distance):
    Patient_idx Patient_ID  success  distance  prob_high  \
33          748      P0807     True  0.000272     0.9575   
48          794      P0906     True  0.000288     0.9825   
50           15      P0900     True  0.000306     1.0000   
11          954      P0783     True  0.000324     0.9975   
35          146      P0811     True  0.000330     1.0000   
12          515      P0759     True  0.000355     1.0000   
6           682      P0763     True  0.000386     1.0000   
18          412      P0782     True  0.000390     1.0000   
42          821      P0743     True  0.000395     0.9850   
39          646      P0968     True  0.

,Patient_idx,Patient_ID,success,distance,prob_high,deltas,new_values
33,748,P0807,True,0.000272,0.9575,"{'Respiratory_Rate': -0.00023043175367121194, ...","{'Respiratory_Rate': 25.99976956824633, 'Oxyge..."
48,794,P0906,True,0.000288,0.9825,"{'Respiratory_Rate': 8.286728228412699e-05, 'O...","{'Respiratory_Rate': 24.000082867282284, 'Oxyg..."
50,15,P0900,True,0.000306,1.0000,"{'Respiratory_Rate': 0.0002761937926294422, 'O...","{'Respiratory_Rate': 29.00027619379263, 'Oxyge..."
11,954,P0783,True,0.000324,0.9975,"{'Respiratory_Rate': 1.8793292113628013e-06, '...","{'Respiratory_Rate': 31.00000187932921, 'Oxyge..."
35,146,P0811,True,0.000330,1.0000,"{'Respiratory_Rate': -0.00013589901475086208, ...","{'Respiratory_Rate': 24.99986410098525, 'Oxyge..."
12,515,P0759,True,0.000355,1.0000,"{'Respiratory_Rate': -0.00016629023858882874, ...","{'Respiratory_Rate': 30.99983370976141, 'Oxyge..."
6,682,P0763,True,0.000386,1.0000,"{'Respiratory_Rate': 1.1491837504706837e-05, '...","{'Respiratory_Rate': 29.000011491837505, 'Oxyg..."
18,412,P0782,True,0.000390,1.0000,"{'Respiratory_Rate': 0.00019559868262319924, '...","{'Respiratory_Rate': 27.000195598682623, 'Oxyg..."
42,821,P0743,True,0.000395,0.9850,"{'Respiratory_Rate': 0.00033127320364201296, '...","{'Respiratory_Rate': 30.000331273203642, 'Oxyg..."
39,646,P0968,True,0.000418,0.9650,"{'Respiratory_Rate': -4.499700408899798e-05, '...","{'Respiratory_Rate': 25.99995500299591, 'Oxyge..."


,Patient_idx,Patient_ID,current_high_prob,future_high_prob_mean,future_high_prob_p95,future_high_prob_p05,future_high_prob_std
0,507,P0384,0.0000,0.091475,0.488250,0.000000,0.157974
1,818,P0096,0.0000,0.033288,0.215000,0.000000,0.070749
2,452,P0394,0.0175,0.142150,0.483000,0.005000,0.163333
3,368,P0389,0.0000,0.078713,0.352500,0.000000,0.138623
4,242,P0515,0.0025,0.202075,0.682750,0.000000,0.229737
5,929,P0898,1.0000,0.999125,1.000000,0.992500,0.002288
6,262,P0098,0.0000,0.037075,0.228000,0.000000,0.090285
7,810,P0541,0.0250,0.431450,0.890375,0.052375,0.277020
8,318,P0729,0.9950,0.998525,1.000000,0.995000,0.002003
9,49,P0838,1.0000,1.000000,1.000000,1.000000,0.000000


,Patient_idx,Patient_ID,success,distance,prob_high,deltas,new_values,current_high_prob,future_high_prob_mean,future_high_prob_p95,future_high_prob_p05,future_high_prob_std,Risk_Drift_Score,Current_Risk
0,37,P0585,True,2.018964,0.5025,"{'Respiratory_Rate': 0.7001885051061478, 'Oxyg...","{'Respiratory_Rate': 30.700188505106148, 'Oxyg...",NaN,NaN,NaN,NaN,NaN,3.9,Medium
1,726,P0123,False,4.594166,0.1200,"{'Respiratory_Rate': 0.7631443970285723, 'Oxyg...","{'Respiratory_Rate': 16.763144397028572, 'Oxyg...",NaN,NaN,NaN,NaN,NaN,3.9,Normal
2,846,P0574,True,5.700781,0.5175,"{'Respiratory_Rate': 1.500064872899145, 'Oxyge...","{'Respiratory_Rate': 21.500064872899145, 'Oxyg...",NaN,NaN,NaN,NaN,NaN,3.9,Medium
3,295,P0469,True,2.590421,0.5025,"{'Respiratory_Rate': 0.5000142916707162, 'Oxyg...","{'Respiratory_Rate': 20.500014291670716, 'Oxyg...",NaN,NaN,NaN,NaN,NaN,3.9,Medium
4,924,P0428,True,2.397282,0.5000,"{'Respiratory_Rate': 1.5405295608311178, 'Oxyg...","{'Respiratory_Rate': 24.540529560831118, 'Oxyg...",NaN,NaN,NaN,NaN,NaN,1.2,Medium
5,658,P0163,True,7.622283,0.5400,"{'Respiratory_Rate': 2.500245668455136, 'Oxyge...","{'Respiratory_Rate': 25.500245668455136, 'Oxyg...",NaN,NaN,NaN,NaN,NaN,3.9,Low
6,682,P0763,True,0.000386,1.0000,"{'Respiratory_Rate': 1.1491837504706837e-05, '...","{'Respiratory_Rate': 29.000011491837505, 'Oxyg...",NaN,NaN,NaN,NaN,NaN,3.9,High
7,286,P0360,True,9.729927,0.5000,"{'Respiratory_Rate': -0.6580401595558065, 'Oxy...","{'Respiratory_Rate': 18.341959840444193, 'Oxyg...",NaN,NaN,NaN,NaN,NaN,3.9,Low
8,880,P0041,False,4.357793,0.1225,"{'Respiratory_Rate': 0.6587028054110888, 'Oxyg...","{'Respiratory_Rate': 15.658702805411089, 'Oxyg...",NaN,NaN,NaN,NaN,NaN,3.9,Normal
9,272,P0901,True,0.000436,0.9700,"{'Respiratory_Rate': -2.9167445248390322e-05, ...","{'Respiratory_Rate': 32.99997083255475, 'Oxyge...",NaN,NaN,NaN,NaN,NaN,1.2,High


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=bdee0b07-dd93-46b8-8522-cce134092e71' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>